In [1]:
from torch.utils.data import DataLoader
from torch import optim
from torch.backends import cudnn
import numpy as np
import time
import csv
import warnings

from lib.aperture_dataset import ApertureDataset
from lib.lenet import LeNet
from lib.fit import fit

log_fname = 'results_%s.csv' % time.strftime('%Y%m%d-%H%M%S')

k = 3
fname_train = 'training_data/training_data.h5'
fname_validate = 'training_data/validation_data.h5'
save_path = 'save_here'


num_samples = 10 ** 5
dat_train = ApertureDataset(fname_train, num_samples, k)

num_samples = 10 ** 4
dat_train2 = ApertureDataset(fname_train, num_samples, k)

num_samples = 10 ** 4
dat_validate = ApertureDataset(fname_validate, num_samples, k)


/Users/zhanwenchen/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# TODO: Implement Dropout"
# TODO: Try Adam

using_cuda = False
if not using_cuda: warnings.warn("Not using CUDA")
if using_cuda: cudnn.benchmark = True # CHANGED

/Users/zhanwenchen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Not using CUDA
  """


In [3]:
# performance
num_workers = 1 # num_workers = 0 is really slow - 12-14 per epoch with batch_size = 32. num_workers = 1 is good (8-10). num_workers = 2 is ok (9-11)
pin_memory = True

batch_size = 32

# NOTE we are using early stopping where we stop training once validation cost stops decreasing.
train_loader = DataLoader(dat_train, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory) # CHANGED
train_loader2 = DataLoader(dat_train2, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory) # CHANGED
validate_loader = DataLoader(dat_validate, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory) # CHANGED

# Model parameters
input_size = 65
output_size = 130


In [4]:
# Train "best model"
learning_rate = 0.05
optimization_algo = "Momentum"
if optimization_algo == "Momentum": momentum = 0.9

conv1_kernel_size = 8
conv1_num_kernels = 23
conv1_stride = 1

pool1_kernel_size = 2 # try 3
pool1_stride = 2

conv2_kernel_size = 4
conv2_num_kernels = 26
conv2_stride = 1

pool2_kernel_size = 2
pool2_stride = 2

fcs_hidden_size = 128 # try 128, 256
fcs_num_hidden_layers = 2


model = LeNet(input_size, output_size, fcs_hidden_size, fcs_num_hidden_layers,
              pool1_kernel_size,
              conv1_kernel_size, conv1_num_kernels, conv1_stride,
              pool2_kernel_size,
              conv2_kernel_size, conv2_num_kernels, conv2_stride)

if using_cuda: model.cuda()
optimizer = optim.SGD(model.parameters(), learning_rate, momentum=momentum)

best_valid_cost, diff = fit(model, train_loader, train_loader2, validate_loader, optimizer, save_path, cuda=using_cuda)



E: 01 / Train: 3.947e-16 / Valid: 3.947e-16 / Diff Valid: -100.00% / Diff Valid-Train: 0.0% / Time: 15.62
E: 02 / Train: 3.947e-16 / Valid: 3.947e-16 / Diff Valid: -0.00% / Diff Valid-Train: 0.0% / Time: 16.40
E: 03 / Train: 3.947e-16 / Valid: 3.947e-16 / Diff Valid: 0.00% / Diff Valid-Train: 0.0% / Time: 15.64
E: 04 / Train: 3.947e-16 / Valid: 3.947e-16 / Diff Valid: 0.00% / Diff Valid-Train: 0.0% / Time: 15.26
E: 05 / Train: 3.947e-16 / Valid: 3.947e-16 / Diff Valid: 0.00% / Diff Valid-Train: 0.0% / Time: 14.99
E: 06 / Train: 3.934e-16 / Valid: 3.934e-16 / Diff Valid: -0.31% / Diff Valid-Train: 0.0% / Time: 14.62
E: 07 / Train: 3.934e-16 / Valid: 3.934e-16 / Diff Valid: 0.00% / Diff Valid-Train: 0.0% / Time: 15.02
E: 08 / Train: 3.934e-16 / Valid: 3.934e-16 / Diff Valid: 0.00% / Diff Valid-Train: 0.0% / Time: 14.43
E: 09 / Train: 3.934e-16 / Valid: 3.934e-16 / Diff Valid: 0.00% / Diff Valid-Train: 0.0% / Time: 14.88
E: 10 / Train: 3.934e-16 / Valid: 3.934e-16 / Diff Valid: 0.00% / Di

Process Process-46:
Traceback (most recent call last):
  File "/Users/zhanwenchen/anaconda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/zhanwenchen/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/zhanwenchen/anaconda/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
  File "/Users/zhanwenchen/anaconda/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/Users/zhanwenchen/anaconda/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/Users/zhanwenchen/anaconda/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/Users/zhanwenchen/anaconda/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remain

KeyboardInterrupt: 

In [ ]:
# # TODO: Implement Dropout
# # optimization parameters
# # try_learning_rates = [1.0, 0.75, 0.5, 0.25, 0.1, 0.05, 0.01, 0.005, 0.001]
# learning_rate = 0.05
# optimization_algo = "Momentum"
# if optimization_algo == "Momentum": momentum = 0.9
# # try_batch_sizes = [32, 64, 128]
# batch_size = 32



# # conv1
# try_conv1_kernel_sizess = list(range(6, 10)) # CHANGED
# try_conv1_num_kernels = list(range(16, 41)) # CHANGED
# # try_conv1_strides = [1, 2] # CHANGED
# conv1_stride = 1

# # pool1
# # try_pool1_sizes = [2, 3]
# pool1_kernel_size = 3
# pool1_stride = 2

# # conv2
# # TODO in loop: conv2_kernel_size should <= conv1_kernel_size
# # TODO in loop: conv2_num_kernels should >= conv1_num_kernels
# try_conv2_kernel_sizess = list(range(2, 10))
# try_conv2_num_kernels = list(range(2, 33))
# # try_conv2_strides = [1, 2]
# conv2_stride = 1

# # pool2
# # try_pool2_sizes = [2, 3]
# pool2_kernel_size = 2
# pool2_stride = 2

# # fcs
# # try_fc_hidden_sizes = [16, 32, 64, 128, 256, 512] # CHANGED: eliminated 8, 1024
# try_fc_hidden_sizes = list(range(128, 386, 2))
# try_fc_num_hidden_layers = [2, 3]
                    

# # Random Hnv1_output_size[1], float) and not conv1_output_size[1].is_integer():
# #         # conv1_output_size must be an integer
# #         # random search is not the most efficient approach but I'm too lazy to filter right now.
# #         conv1_stride = choose_int(try_conv1_strides)
# #         conv1_output_size = yperparameter Search
# def choose_int(array): return int(np.random.choice(array)) # int() because PyTorch doesn't convert np.int64 to int.

# num_trys = 100
# run_count = 0

# print()
# print("| #  | best valid cost | diff% | learn_rate | batch_size | conv1_kernel_size | conv1_num_kernels | conv1_stride | pool1_kernels | conv2_kernel_size | conv2_num_kernels | conv2_strides | pool2_kernels | fcs_size | fcs_num_hidden_layers |")
# print("| -- | --------------- | ----- | ---------- | ---------- | ----------------- | ----------------- | ------------ | ------------- | ----------------- | ----------------- | ------------- | ------------- | -------- | --------------------- |")

# while run_count < num_trys:
#     # choose random hyperparameters: optimization
# #     batch_size = choose_int(try_batch_sizes)
# #     learning_rate = np.random.choice(try_learning_rates)
    
#     # choose random hyperparameters: model
#     conv1_kernel_size = choose_int(try_conv1_kernel_sizess)
#     conv1_num_kernels = choose_int(try_conv1_num_kernels)
# #     conv1_stride = choose_int(try_conv1_strides)
    
#     # enforce relative shape and divisibility
#     conv1_output_size = (conv1_num_kernels, (input_size - conv1_kernel_size) / conv1_stride + 1)
# #     while isinstance(conv1_output_size[1], float) and not conv1_output_size[1].is_integer():
# #         # conv1_output_size must be an integer
# #         # random search is not the most efficient approach but I'm too lazy to filter right now.
# #         conv1_stride = choose_int(try_conv1_strides)
# #         conv1_output_size = (conv1_num_kernels, (input_size - conv1_kernel_size) / conv1_stride + 1)
    
#     # pool1_kernel_size = 3 by default
#     if conv1_output_size[1] % 2 == 0: pool1_kernel_size = 2
#     pool1_output_size = (conv1_num_kernels, (conv1_output_size[1] - pool1_kernel_size) / pool1_stride + 1)
# #     while isinstance(pool1_output_size[1], float) and not pool1_output_size[1].is_integer():
# #         # conv1_output_size must be an integer
# #         pool1_kernel_size = choose_int(try_pool1_sizes)
# #         pool1_output_size = (conv1_num_kernels, (conv1_output_size[1] - pool1_kernel_size) / pool1_stride + 1)
    
#     conv2_kernel_size = choose_int(try_conv2_kernel_sizess)
#     conv2_num_kernels = choose_int(try_conv2_num_kernels)
# #     conv2_stride = choose_int(try_conv2_strides)
    
#     conv2_output_size = (conv2_num_kernels, (pool1_output_size[1] - conv2_kernel_size) / conv2_stride + 1)
# #     while isinstance(conv2_output_size[1], float) and not conv2_output_size[1].is_integer():
# #         # conv2_output_size must be an integer
# #         conv2_stride = choose_int(try_conv2_strides)
# #         conv2_output_size = (conv2_num_kernels, (pool1_output_size[1] - conv2_kernel_size) / conv2_stride + 1)

#     # by default pool2_kernel_size = 2
#     if conv2_output_size[1] % 2 == 1: pool2_kernel_size = 3
#     pool2_output_size = (conv2_num_kernels, (conv2_output_size[1] - pool2_kernel_size) / pool2_stride + 1)
    
# #     while isinstance(pool2_output_size[1], float) and not pool2_output_size[1].is_integer():
# #         # conv1_output_size must be an integer
# #         pool2_kernel_size = choose_int(try_pool2_sizes)
# #         pool2_output_size = (conv2_num_kernels, (conv2_output_size[1] - pool2_kernel_size) / pool2_stride + 1)
    
#     fcs_hidden_size = choose_int(try_fc_hidden_sizes)
#     fcs_num_hidden_layers = choose_int(try_fc_num_hidden_layers)
    
#     # load data because of difference batch_sizes
# #     train_loader = DataLoader(dat_train, batch_size=batch_size, shuffle=True, num_workers=1, pin_memory=False) # CHANGED
# #     train_loader2 = DataLoader(dat_train2, batch_size=batch_size, shuffle=False, num_workers=1, pin_memory=False) # CHANGED
# #     validate_loader = DataLoader(dat_validate, batch_size=batch_size, shuffle=False, num_workers=1, pin_memory=False) # CHANGED
    
# #     try:
#     # build the model and train
#     model = LeNet(input_size, output_size, fcs_hidden_size, fcs_num_hidden_layers,
#                   batch_size,
#                   pool1_kernel_size,
#                   conv1_kernel_size, conv1_num_kernels, conv1_stride,
#                   pool2_kernel_size,
#                   conv2_kernel_size, conv2_num_kernels, conv2_stride)

#     if using_cuda: model.cuda()
#     optimizer = optim.SGD(model.parameters(), learning_rate, momentum=momentum)

#     best_valid_cost, diff = fit(model, train_loader, train_loader2, validate_loader, optimizer, save_path, cuda=using_cuda)


#     print("| %02d | %.3E | %0.3f | %0.7f  | %d        | %02d                | %02d                 | %d            | %d             | %d                | %d             | %d             | %d           | %04d     | %s |" % \
#           (run_count,best_valid_cost, diff,learning_rate,batch_size,conv1_kernel_size,  conv1_num_kernels,      conv1_stride,   pool1_kernel_size,    conv2_kernel_size,  conv2_num_kernels,   conv2_stride, pool2_kernel_size,    fcs_hidden_size, fcs_num_hidden_layers))
#     run_count += 1
# #     except:
# #         print("| %02d | error | error | %0.7f  | %d        | %02d                | %02d                 | %d            | %d             | %d                | %d             | %d             | %d           | %04d     | %s |" % \
# #               (run_count,learning_rate,batch_size,conv1_kernel_size,  conv1_num_kernels,      conv1_stride,   pool1_kernel_size,    conv2_kernel_size,  conv2_num_kernels,   conv2_stride, pool2_kernel_size,    fcs_hidden_size, fcs_num_hidden_layers))
        

# print("\nIt's all over.")